In [5]:
import requests
import re
import json
import pandas as pd
from pprint import pprint
from time import sleep

from spotify_credentials import client_id, client_secret

In [6]:
#getting spotify access token
url = "https://accounts.spotify.com/api/token"
header = {"Content-Type": "application/x-www-form-urlencoded"}
data = {"grant_type":"client_credentials",
        "client_id" : client_id,
        "client_secret" : client_secret}

r = requests.post(url, data=data, headers=header).json()

spotify_api_token = r['access_token']

In [7]:
songdf = pd.read_csv(r"resources\most_frequent_hot100songs_by_year\top100_2024.csv")
songdf["id"] = ""
#songdf.head()
songdf_copy = songdf.copy()


In [10]:
#This block of code iterates over the dataframe songdf_copy set above. At each row of the DataFrame# 
#it gets artist and song values and formats them, then passes these values into the Spotify search endpoint (https://developer.spotify.com/documentation/web-api/reference/search), which returns a unique song ID.
#This song id value is assigned to the [row, id] position, for the current row.
#a csv with song IDs added to each row is created.

#setting Spotify endpoint header
spotify_headers = {'Authorization': 'Bearer {}'.format(spotify_api_token)}


for index, row in songdf.iterrows():#https://www.geeksforgeeks.org/iterating-over-rows-and-columns-in-pandas-dataframe/

    #value = songdf.iloc[index]["title"]
    #print(value)
    

    artist = re.split(' Featuring | With | Duet | X | x | & |, | \\(| And | Feat.', row[2])[0].split()
    artist_final = '+'.join(artist)

    song = re.split(' \\(', row[1])[0].replace("/"," ").split()
    song_final = '+'.join(song)

    try:
        search_url = f"https://api.spotify.com/v1/search?q=track%3A{song_final}+artist%3A{artist_final}&type=track"
        search_response = requests.get(search_url, headers = spotify_headers).json()
        song_id = search_response["tracks"]["items"][0]["id"]
        #print(f"{song_final} by {artist_final}: {song_id}")
        

    except:
        print(f"no ID available for song named: {song_final} by {artist_final}")
        pass


    songdf_copy.iloc[index]["id"] = str(song_id)

print("---------------")
print("calls complete.")
print("---------------")

songdf_copy.to_csv("resources\top100_2024_withID.csv", index=False)

#path = r"C:\Users\kevin\bootcamp\Project_1\resources"
#songdf_copy.to_csv(os.path.join(path,f"top100_2024_withID.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas

C:\Users\kevin\AppData\Local\Temp\ipykernel_16196\2948875631.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  artist = re.split(' Featuring | With | Duet | X | x | & |, | \\(| And | Feat.', row[2])[0].split()
C:\Users\kevin\AppData\Local\Temp\ipykernel_16196\2948875631.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  song = re.split(' \\(', row[1])[0].replace("/"," ").split()
C:\Users\kevin\AppData\Local\Temp\ipykernel_16196\2948875631.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

no ID available for song named: Carnival by ¥$:+Kanye+West
---------------
calls complete.
---------------


OSError: [Errno 22] Invalid argument: 'resources\top100_2024_withID.csv'